# Poetry Notebook

In this notebook we will be implementing GPT to generate text based on the work of Edgar Allan Poe.

In [1]:
# Installing dependencies
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.7 MB/s eta 0:00:00


In [2]:
# Downloading dataset from the GitHub
!wget https://raw.githubusercontent.com/kocenko/Poetry-Synthesis/dev/data/poe_data.txt

--2023-05-16 06:10:37--  https://raw.githubusercontent.com/kocenko/Poetry-Synthesis/dev/data/poe_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1930488 (1.8M) [text/plain]
Saving to: ‘poe_data.txt’

poe_data.txt        100%[===================>]   1.84M  --.-KB/s    in 0.007s  

2023-05-16 06:10:37 (260 MB/s) - ‘poe_data.txt’ saved [1930488/1930488]



In [3]:
# Essential imports

import tiktoken

In [4]:
# Reading and file and displaying number of characters
with open("poe_data.txt", encoding="utf-8") as f:
  text = f.read()

print("Number of characters in the set: {}".format(len(text)))

Number of characters in the set: 1905067


In [10]:
# Setting up encoder/decoder

enc = tiktoken.get_encoding("cl100k_base")
encoded = enc.encode(text[:10])
decoded = enc.decode(encoded)
assert text[:10] == decoded, "There is some problem with tiktoken tokenizer"